In [1]:
import tensorflow as tf
tf.enable_eager_execution()


import numpy as np
import os
import time

C:\Users\Isaiah\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
text=open('lyrics.txt', encoding="utf8").read()
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

def generate_text(model, start_string, num_chars):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = num_chars

  # You can change the start string to experiment
  #start_string = 'ROMEO'

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.75

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 512

rnn = tf.keras.layers.CuDNNGRU

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      tf.keras.layers.Dropout(rate=0.5),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(vocab_size),
  ])
  return model

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights('C:\\Users\\Isaiah\\python_projects\\rapgenius\\checkpoints_v3\\ckpt_40')

model.build(tf.TensorShape([1, None]))

In [29]:
print(generate_text(model, start_string='"What', num_chars=1000))

"What? What you think about the pride?

[Chorus: Kerdrick Lamar]
I'm rollin' up in the morning (yeah)
Your girl in it (ay yeah)
If you ain't losing you guilan (hey)
The boy just let me give you under to a ha
So make your pistol hate me how he can do to me

[Verse 2]
I couldn't even feel the call of life
I spin him and then I can see a right of ece
The thug niggas don't be like for nothing
They say "You can't be like "What?"
I see you just a motherfucking gaabers
Remember what I'm making in your thing
You can see me down up and asking me
I mean you ain't wanna get down
Let me get down piece out and have the story
I'm on the real nigga, cause I'm tired of one
I'm a nigga bitch, I'm a nigga, I'm doing the same crew
Smiling on the streets of a big famori
That ain't going for that nigga
While it happens on my fan-ass
And these niggas barked in the shit
I don't know what I mean
That's how them white motherfuckers

[Hook]
In the ride
This right the way that you say, I'm a good night
I ain't w